In [10]:
import numpy as np
from scipy.interpolate import interp1d, interp2d
import math
import scipy.io as scio
from keras.models import load_model
import pandas as pd

In [ ]:
def CalcTorque(v_k,GP,a):
    g = 9.81                            #Gravity, m/s^2
    m= 1750                             #Vehicle mass, Kg
    f = 0.01                            #Rolling coefficient, N·m
    C_d = 0.26
    A = 2.33
    R_roll = 0.3
    v=v_k/3.6

    theta=math.atan(GP/100)
    w_whl=v/R_roll

    T_whl=R_roll*(m*a+m*g*f*math.cos(theta)+m*g*math.sin(theta)+(C_d*A*v^2)/21.15)
    P_req=T_whl*w_whl

    return(P_req, T_whl, w_whl)
    
def ICEparams(Ksplit,P_req,T_whl):
    P_ice=P_req*Ksplit
    Eng_pwr_opt_list = np.arange(0, 57, 1) * 1000
    W_list = [91.106186954104, 83.77580409572782, 83.77580409572782, 83.77580409572782, 83.77580409572782, 83.77580409572782, 83.77580409572782, 89.0117918517108, 94.24777960769379, 117.28612573401894, 121.47491593880532, 132.9940890019679, 142.4188669627373, 145.56045961632708, 168.59880574265222, 170.69320084504542, 181.1651763570114, 183.25957145940458, 183.25957145940458, 192.684349420174, 204.20352248333657, 210.48670779051614, 215.72269554649912, 220.95868330248211, 248.18581963359367, 237.71384412162766, 255.51620249196986, 260.7521902479528, 264.9409804527392, 269.1297706575256, 274.3657584135086, 281.69614127188476, 289.026524130261, 294.262511886244, 298.4513020910303, 301.59289474462014, 305.78168494940655, 309.9704751541929, 314.1592653589793, 320.4424506661589, 328.8200310757317, 336.15041393410786, 344.52799434368063, 352.90557475325346, 361.2831551628262, 369.660735572399, 378.0383159819718, 385.368698840348, 394.7934768011173, 402.12385965949346, 410.5014400690663, 418.87902047863906, 427.2566008882119, 436.6813788489813, 442.96456415616086, 451.3421445657336, 459.7197249753064]
    Eng_pwr_func = interp1d(Eng_pwr_opt_list, W_list)  
    w_ice0 = Eng_pwr_func(P_ice)

    T_ice0=P_ice/w_ice0

    if T_whl>0:
        T_ice=T_ice0
        w_ice=w_ice0
    else:
        T_ice=0
        w_ice=0
    
    return (T_ice,w_ice)

def EMx_pwr(T_whl, w_whl, T_ice, w_ice,Ksplit):
    k1=1.92
    k2=2.6
    kf=4.1

    if Ksplit==0:
        w_ice=0
        w_EM1=(k1+1)*kf*w_whl
        w_EM2=-k2*kf*w_whl
        T_EM2=-T_whl/(k2*kf)
        T_EM1=0
    else:
        T_EM2 = 1 / (k1 + k2 + 1) * (T_whl * k1 / kf - (k1 + 1) * T_ice)
        T_EM1 = 1 / k1 * T_ice + (k2 + 1) / k1 * T_EM2
        w_C1R2 = kf * w_whl
        w_EM1 = (k1 + 1) * w_C1R2 - k1 * w_ice
        w_EM2 = (k1 + k2 + 1) / k1 * w_C1R2 - (k2 + 1) / k1 * w_C1R2
    
    P_EM1 = w_EM1 * T_EM1 * 0.001
    P_EM2 = w_EM2 * T_EM2 * 0.001
    n_ice=w_ice*2*math.pi/60

    return(P_EM1, P_EM2,n_ice)

def BSFC(w_ice,T_ice):
    #BSFC Implementation
    m_inj=0.2
    Eng_spd_list = np.arange(0, 4501, 125) * (2 * math.pi) / 60
    Eng_spd_list = Eng_spd_list[np.newaxis, :]
    Eng_trq_list = np.arange(0, 111, 5) * (121 / 110)
    Eng_trq_list = Eng_trq_list[np.newaxis, :]
    data_path = 'Eng_bsfc_map.mat'
    data = scio.loadmat(data_path)
    Eng_bsfc_map = data['Eng_bsfc_map']    
    Eng_fuel_map = Eng_bsfc_map * (Eng_spd_list.T * Eng_trq_list) / 3600 / 1000
    Eng_fuel_func = interp2d(Eng_trq_list, Eng_spd_list, Eng_fuel_map)
    m_inj = Eng_fuel_func(T_ice, w_ice)

    return m_inj

def NOx2gs(NO,NO2):
    PM_NO=30006.1
    PM_NO2=46005.5
    NOx_gs=NO*PM_NO+NO2*PM_NO2

    return NOx_gs

def scalerin_model(ScalerMax,ScalerMin,A):
    max=1
    min=-1
    A_sc=np.ones(A.shape)
    for i in A.shape[2]:
        X_std=(A[0,0,i]-ScalerMin[i])/(ScalerMax[i]-ScalerMin[i])
        X_scaled=X_std*(max-min)+min
        A_sc[0,0,i]=X_scaled

    return A_sc

def scalerout_model(ScalerMax,ScalerMin,A):
    max=1
    min=-1
    A_sc=np.ones(A.shape)
    for i in A.shape[2]:
        X_std=(A[0,0,i]-min)/(max-min)
        X_scaled=X_std*(ScalerMax[i]-ScalerMin[i])+ScalerMin[i]
        A_sc[0,0,i]=X_scaled

    return A_sc

def scalerin_SAC(ScalerMax,ScalerMin,A):
    max=1
    min=0
    A_sc=np.ones(A.shape)
    for i in A.shape[2]:
        X_std=(A[0,0,i]-ScalerMin[i])/(ScalerMax[i]-ScalerMin[i])
        X_scaled=X_std*(max-min)+min
        A_sc[0,0,i]=X_scaled

    return A_sc

def scalerout_SAC(ScalerMax,ScalerMin,A):
    max=1
    min=0
    A_sc=np.ones(A.shape)
    for i in A.shape[2]:
        X_std=(A[0,0,i]-min)/(max-min)
        X_scaled=X_std*(ScalerMax[i]-ScalerMin[i])+ScalerMin[i]
        A_sc[0,0,i]=X_scaled

    return A_sc

In [ ]:
model = load_model('model.h5')

In [14]:
pathscalers=r"C:\Users\fjcub\Documents\UPC\INIREC\CORNET\Simulink\SAC\scalers2py.mat"
scalersmat=scio.loadmat(pathscalers)
ScalerInMax=scalersmat['ScalerInMax'].flatten()
ScalerInMin=scalersmat['ScalerInMin'].flatten()
ScalerOut_range=scalersmat['ScalerOut_range'].flatten()
ScalerSACMax=scalersmat['ScalerSACMax'].flatten()
ScalerSACMin=scalersmat['ScalerSACMin'].flatten()
ScalerOutMax=scalersmat['ScalerOutMax'].flatten()
ScalerOutMin=scalersmat['ScalerOutMin'].flatten()

150.603


In [ ]:
# Leer el archivo CSV
csvpath=r"C:\Users\fjcub\Documents\UPC\INIREC\CORNET\Simulink\SAC\SAC_DC_2py.csv"
df = pd.read_csv(csvpath, header=None)

# Asignar nombres a las columnas
df.columns = ['Time', 'vk_ref', 'GP_ref', 'a_ref']

v_kvect=np.array(df['vk_ref'])
GPvect=np.array(df['GP_ref'])
accvect=np.array(df['a_ref'])

SoC_ref=0.5
dist = np.sum(v_kvect) / 3600
NOx_ref = 0.8 * dist / len(v_kvect)

main_in = np.ones((1,1,6))
aux_in = np.ones((1,1,4))
d_aux_in=np.ones((1,1,4))

for i in d_aux_in.shape[2]:
    d_aux_in[0,0,i]=ScalerOut_range[i]/2+ScalerOutMin[i]
    

[ 0.      0.      0.     ... -1.3889 -1.1111 -1.1111]


In [ ]:
c=1 #Indicador bucle
v_k=v_kvect[c]
GP=GPvect[c]
acc=accvect[c]

P_req, T_whl, w_whl = CalcTorque(v_k,GP, acc)
T_ice,w_ice = ICEparams(Ksplit,P_req,T_whl)
P_EM1, P_EM2,n_ice = EMx_pwr(T_whl, w_whl, T_ice, w_ice,Ksplit)
m_inj = BSFC(w_ice,T_ice)

main_in[0,0,0] = v_k   # Velocity_kph
main_in[0,0,1] = n_ice # Engine_Speed_RPM similar to set point
main_in[0,0,2] = P_EM1  # EM1 Power similar to set point
main_in[0,0,3] = P_EM1  # EM2 Power similar to set point
main_in[0,0,4] = GP   # Grade in percentage
main_in[0,0,5] = m_inj    # Injection mass similar to set point

if step==0:
    aux_in=d_aux_in
    
main_out = model.predict([main_in, aux_in], batch_size=1, steps=1)
aux_in=main_out

main_out_real = scalerout_model(ScalerOutMin,ScalerOutMax,main_out)

SACmat=np.ones((1,1,7))
SACmat[0,0,0] = v_k   # Velocity_kph
SACmat[0,0,1] = main_out_real[0,0,0]
SACmat[0,0,2] = main_out_real[0,0,1]
SACmat[0,0,3] = main_out_real[0,0,2]
SACmat[0,0,4] = NOx2gs(main_out_real[0,0,1],main_out_real[0,0,2])  
SACmat[0,0,5] = SoC_ref
SACmat[0,0,5] = NOx_ref


SACmat_sc=scalerin_SAC(ScalerSACMin,ScalerSACMax,SACmat)
vk_sc=SACmat_sc[0,0,0]
SoC_sc=SACmat_sc[0,0,1]
NO_sc=SACmat_sc[0,0,2]
NO2_sc=SACmat_sc[0,0,3]
NOx_sc=SACmat_sc[0,0,4]
SoCref_sc=SACmat_sc[0,0,5]
NOxref_sc=SACmat_sc[0,0,6]

obsv=[vk_sc,SoC_sc,NO_sc,NO2_sc]
c1=0.3
c2=0.7
reward = c1*(SoC_sc-SoCref_sc)^2 - c2*(NOx_sc-NOxref_sc)^2





In [9]:
A=np.ones((1,1,6))
print(A.shape)
B=np.ones(A.shape)
print(A[0,0,0])

(1, 1, 6)
1.0
